# U.S. Opiate Prescriptions

Accidental death by fatal drug overdose is a rising trend in the United States. What can you do to help?

This dataset contains summaries of prescription records for 250 common opioid and non-opioid drugs written by 25,000 unique licensed medical professionals in 2014 in the United States for citizens covered under Class D Medicare as well as some metadata about the doctors themselves. This is a small subset of data that was sourced from cms.gov. The full dataset contains almost 24 million prescription instances in long format. I have cleaned and compiled this data here in a format with 1 row per prescriber and limited the approximately 1 million total unique prescribers down to 25,000 to keep it manageable. If you are interested in more data, you can get the script I used to assemble the dataset here and run it yourself. The main data is in prescriber-info.csv. There is also opioids.csv that contains the names of all opioid drugs included in the data and overdoses.csv that contains information on opioid related drug overdose fatalities.

### A key for reading the dataset

NPI – unique National Provider Identifier number

Gender - (M/F)

State - US State by abbreviation

Credentials - set of initials indicative of medical degree

Specialty - description of type of medicinal practice

Opioid.Prescriber - a boolean label indicating whether or not that individual prescribed opiate drugs more than 10 times in the year

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
opioids = pd.read_csv('opioids.csv')
overdoses = pd.read_csv('overdoses.csv')
prescriber = pd.read_csv('prescriber-info.csv')

# Exploratory Analysis

In [3]:
# See what the we're working with for these dataframes
opioids.head()

,Drug Name,Generic Name
0,ABSTRAL,FENTANYL CITRATE
1,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE
2,ACTIQ,FENTANYL CITRATE
3,ASCOMP WITH CODEINE,CODEINE/BUTALBITAL/ASA/CAFFEIN
4,ASPIRIN-CAFFEINE-DIHYDROCODEIN,DIHYDROCODEINE/ASPIRIN/CAFFEIN


In [6]:
overdoses.head()

,State,Population,Deaths,Abbrev
0,Alabama,"4,833,722",723,AL
1,Alaska,"735,132",124,AK
2,Arizona,"6,626,624","1,211",AZ
3,Arkansas,"2,959,373",356,AR
4,California,"38,332,521","4,521",CA


In [7]:
prescriber.head()

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1
2,1427182161,F,NY,M.D.,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
3,1669567541,M,AZ,MD,Internal Medicine,0,43,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1679650949,M,NV,M.D.,Hematology/Oncology,0,0,0,0,0,...,0,0,0,0,17,28,0,0,0,1


In [63]:
# Find any NaN listings in the Specialty column
prescriber.isnull().sum()

NPI                  0
Gender               0
State                0
Specialty            0
ABILIFY              0
                    ..
ZIPRASIDONE.HCL      0
ZOLPIDEM.TARTRATE    0
Opioid.Prescriber    0
SumOpioids           0
TotPresc             0
Length: 257, dtype: int64

In [30]:
# Since we'll focus on the Specialty column, we'll go ahead and remove the Credentials column
prescriber = prescriber.drop('Credentials', axis=1)

In [13]:
#Running a value count on the State column shows us some extra non US states. We'll drop the non states and add DC into VA
prescriber['State'].value_counts()

# Drop non US States: PR, AA, GU, AE, ZZ
prescriber = prescriber[(prescriber.State != 'GU') & (prescriber.State != 'AA') & (prescriber.State != 'PR') & (prescriber.State != 'AE') & (prescriber.State != 'ZZ') & (prescriber.State != 'VI')]

In [14]:
# Move any rows with DC into VA by renaming DC to VA
prescriber['State'] = prescriber['State'].str.replace('DC', 'VA')

In [23]:
prescriber['State'].value_counts()

CA    2562
NY    1956
FL    1570
TX    1500
PA    1211
IL    1002
OH     981
MI     872
NC     778
MA     725
NJ     649
VA     647
GA     613
WA     578
TN     552
IN     533
AZ     509
MD     509
WI     498
MO     483
MN     448
CO     393
SC     390
CT     388
KY     367
LA     354
OR     344
AL     344
OK     281
IA     225
AR     216
KS     203
WV     199
MS     193
NM     166
UT     162
NV     155
ME     147
NE     137
ID     133
NH     119
RI     117
DE      91
HI      91
SD      83
MT      77
ND      66
VT      65
AK      39
WY      38
Name: State, dtype: int64

# Identifying opioid drug names

Now that we have cleaned up our dataframe a bit we can start looking at the data a bit better. We will start by identifying opioids by comparing drug names found in the prescriber dataframe and drug names found in the opioid dataframe.

In [5]:
#Compare opioids df with prescrition df to find opioids
opi = list(re.sub(r'[-\s]','.',x) for x in opioids.values[:,0])
opi_presc = list(set(opi) & set(prescriber.columns))

opi_presc

['MORPHINE.SULFATE',
 'FENTANYL',
 'HYDROCODONE.ACETAMINOPHEN',
 'OXYCODONE.HCL',
 'METHADONE.HCL',
 'ACETAMINOPHEN.CODEINE',
 'OXYCODONE.ACETAMINOPHEN',
 'MORPHINE.SULFATE.ER',
 'TRAMADOL.HCL',
 'OXYCONTIN',
 'HYDROMORPHONE.HCL']

In [64]:
#Using these 11 opioid drug names we'll create a column with the number of opioids prescribed by each NPI
prescriber['SumOpioids'] = prescriber[opi_presc].sum(axis=1)

#Create a column with the number of total opioids prescribed by each NPI
prescriber['TotPresc'] = (prescriber.iloc[:,5:254]).sum(axis=1)

# Creating Separate Dataframes

In [133]:
#Create variables to easily group certain columns together
state = prescriber['State']
gender = prescriber['Gender']
specialty = prescriber['Specialty']
opi = prescriber[['HYDROMORPHONE.HCL',
 'OXYCONTIN',
 'OXYCODONE.HCL',
 'TRAMADOL.HCL',
 'MORPHINE.SULFATE.ER',
 'HYDROCODONE.ACETAMINOPHEN',
 'FENTANYL',
 'MORPHINE.SULFATE',
 'METHADONE.HCL',
 'OXYCODONE.ACETAMINOPHEN',
 'ACETAMINOPHEN.CODEINE']]
presc = prescriber.iloc[:,5:254]

In [152]:
# Sum of Opioids prescribed by State
df_op_state = opi.groupby(state).sum()

df_op_state.head()

,HYDROMORPHONE.HCL,OXYCONTIN,OXYCODONE.HCL,TRAMADOL.HCL,MORPHINE.SULFATE.ER,HYDROCODONE.ACETAMINOPHEN,FENTANYL,MORPHINE.SULFATE,METHADONE.HCL,OXYCODONE.ACETAMINOPHEN,ACETAMINOPHEN.CODEINE
State,,,,,,,,,,,
AK,84,31,217,118,157,430,41,11,69,87,11
AL,222,716,1641,15264,1100,34560,1738,488,709,4001,920
AR,235,364,2065,5433,1174,16788,941,193,573,2475,580
AZ,728,1678,8526,7284,5101,16095,1436,958,1032,8770,1102
CA,2996,3351,6748,30379,8313,110347,5677,1471,4012,11723,9133


In [120]:
#Opioids prescribed by gender
df_gender_op = opi.groupby(gender).sum()
df_gender_op

,HYDROMORPHONE.HCL,OXYCONTIN,OXYCODONE.HCL,TRAMADOL.HCL,MORPHINE.SULFATE.ER,HYDROCODONE.ACETAMINOPHEN,FENTANYL,MORPHINE.SULFATE,METHADONE.HCL,OXYCODONE.ACETAMINOPHEN,ACETAMINOPHEN.CODEINE
Gender,,,,,,,,,,,
F,6332,16004,54873,113616,25154,232527,26700,6042,11109,59566,13859
M,16827,33723,117413,285118,63036,724912,58644,14665,24680,180568,44618


In [149]:
# Opioids prescribed by specialty
df_spec_op = opi.groupby(specialty).sum()

df_spec_op.head()

,HYDROMORPHONE.HCL,OXYCONTIN,OXYCODONE.HCL,TRAMADOL.HCL,MORPHINE.SULFATE.ER,HYDROCODONE.ACETAMINOPHEN,FENTANYL,MORPHINE.SULFATE,METHADONE.HCL,OXYCODONE.ACETAMINOPHEN,ACETAMINOPHEN.CODEINE
Specialty,,,,,,,,,,,
Addiction Medicine,25,194,820,21,751,334,279,86,559,432,0
Allergy/Immunology,0,0,11,83,0,136,0,0,0,46,0
Anesthesiology,2503,3044,10512,4036,8965,19478,4571,2437,3968,10307,215
Behavioral Analyst,0,0,0,0,0,0,0,0,0,0,0
CRNA,0,0,0,0,0,13,0,0,0,0,0
